## Configurer Apache Airflow avec Docker-Compose 
### 1 . Avant de commencer 

Cette procédure suppose une familiarité avec Docker et Docker Compose.

Docker :  Une plateforme de virtualisation de conteneurs qui permet d'exécuter des applications dans des environnements isolés, appelés conteneurs, afin de faciliter le déploiement et la gestion d'applications sur différents systèmes d'exploitation et infrastructures.

Docker-compose : Est un outil de gestion de conteneurs qui permet de définir et de gérer plusieurs conteneurs Docker en même temps

### 2 . Récupération docker-compose.yaml

Pour le déploiement d'Airflow sur Docker Compose, il est nécessaire de récupérer le fichier docker-compose.yaml

```
curl -LfO 'https://airflow.apache.org/docs/apache-airflow/2.5.1/docker-compose.yaml'
```

Ce fichier contient plusieurs définitions de service:

airflow-init : Le service d'initialisation.

airflow-scheduler : Le planificateur surveilletoutes les tâches et les DAG, puis déclenche le les instances de tâches une fois leurs dépendances terminées.

![image](https://user-images.githubusercontent.com/123757632/225338366-ccd82410-0ea6-44af-98a4-371f537bc1b6.png)

airflow-webserver : Le serveur Web est disponible sur http://localhost:8080.

![image](https://user-images.githubusercontent.com/123757632/225338693-3a69dcf3-0b09-49bd-9ed8-aa739a454a35.png)

airflow-worker : Le travailleur qui exécute les tâches données par le planificateur.

![image](https://user-images.githubusercontent.com/123757632/225338878-5a359306-abd1-4502-bd82-8882184586d7.png)

postgres : La base de données.

![image](https://user-images.githubusercontent.com/123757632/225339186-906134a6-a76e-411f-b621-9339c7d1d87c.png)

redis - Les redis :  courtier qui transmet les messages du planificateur au travailleur.

![image](https://user-images.githubusercontent.com/123757632/225339462-e9f1e990-fda7-471a-885d-4b75f7a3c3dc.png)

### 3 . Initialisation de l'environnement 

Avant de démarrer Airflow pour la première fois, vous devez préparer votre environnement, c'est-à-dire créer les fichiers nécessaires, répertoires et initialiser la base de données.

```
mkdir -p ./dags ./logs ./plugins
echo -e "AIRFLOW_UID=$(id -u)" > .env
```

La première commande crée trois répertoires dans le répertoire actuel (./), nommément dags, logs et plugins. L'option -p signifie que les répertoires parents seront également créés si nécessaire.

La deuxième commande crée un fichier nommé .env dans le répertoire actuel et y écrit une variable d'environnement appelée AIRFLOW_UID contenant l'ID utilisateur de l'utilisateur courant

![image](https://user-images.githubusercontent.com/123757632/225340586-cdca88dd-4977-44e1-b66d-d064c402d9d9.png)

"dags" : C'est un répertoire où vous pouvez stocker des définitions de tâches d'Airflow,qui sont des workflows écrits en Python. Dans ce répertoire, vous pouvez créer un fichier Python pour chaque tâche ou un groupe de tâches liées.

"logs" : Ce répertoire est utilisé par Airflow pour stocker les fichiers de journalisation (logs) générés lors de l'exécution des tâches. 

"plugins" : Ce répertoire est utilisé pour stocker des plugins personnalisés pour Airflow. Les plugins sont des modules Python qui étendent les fonctionnalités d'Airflow, tels que des connecteurs personnalisés ou des opérateurs personnalisés.

### 4 . Initialiser Airflow

Sur tous les systèmes d'exploitation, vous devez exécuter les migrations de base de données et créer le premier compte utilisateur.

```
docker compose up airflow-init
```

Une fois l'initialisation terminée, vous devriez voir un message comme celui-ci:

```
airflow-init_1       | Upgrades done
airflow-init_1       | Admin user airflow created
airflow-init_1       | 2.5.1
start_airflow-init_1 exited with code 0
```

### 5 . Exécution d'un Airflow

Vous pouvez maintenant démarrer tous les services avec :

```
docker compose up -d
```

![image](https://user-images.githubusercontent.com/123757632/230577261-3c692020-4522-49f2-896b-1ca293559dd7.png)

Pour ouvrir un lien dans Gitpod, vous pouvez simplement cliquer sur le lien. Gitpod ouvrira automatiquement le lien dans un nouvel onglet de votre navigateur dans l'environnement Gitpod. 

![image](https://user-images.githubusercontent.com/123757632/225343489-bcaa89aa-9f13-46b7-bbfb-a4e592ee71e6.png)

Vous pouvez vérifier l'état des conteneurs et vous assurer qu'aucun conteneur n'est dans un état malsain:

```
docker compose ps
```

![image](https://user-images.githubusercontent.com/123757632/225344913-b2055e69-8a42-4f40-9162-60d21ec1c784.png)

### L'interface utilisateurs

#### Dag View
  - Liste des DAG de votre environnement et ensemble de raccourcis vers des pages utiles. 
  - Il est possible de visualiser rapidement le nombre précis de tâches réussies, échouées ou en cours d'exécution.

![image](https://user-images.githubusercontent.com/123757632/231165285-6c291fa7-6b8e-4a16-8524-9e9a3c9da8d7.png)

Pour ne pas charger les examples par défaut , veuillez remplacer la valeur true par false dans la ligne 61 du ficher docker-compose.yaml :

```
AIRFLOW__CORE__LOAD_EXAMPLES: 'false'
```

puis réexecuter les commandes : 

```
docker compose up airflow-init
```
et 

```
docker compose up -d
```

![image](https://user-images.githubusercontent.com/123757632/231167184-55e38e52-2229-463a-9bc5-d981d87e4c5a.png)

#### Dataset View 

- Une liste combinée des ensembles de données actuels ainsi qu'un graphique 
illustrant comment ils sont produits et consommés par les DAG.

- En cliquant sur un ensemble de données dans la liste ou le graphique, celui-ci sera mis en évidence ainsi que ses relations, et la liste sera filtrée pour afficher l'historique récent des instances de tâches qui ont mis à jour cet ensemble de données et si cela a déclenché d'autres exécutions de DAG.

![image](https://user-images.githubusercontent.com/123757632/231167619-faccda43-50f4-451f-a699-f886142ca562.png)

#### Grid View 

- Un graphique en barres et une représentation en grille du DAG qui s'étend dans le temps. 
- La rangée supérieure est un graphique des DAG Runs par durée, et en dessous, les instances de tâches. 
- Si un pipeline est en retard, vous pouvez rapidement voir où se situent les différentes étapes et identifier celles qui bloquent.


![image](https://user-images.githubusercontent.com/123757632/231167669-6457209e-cf7e-4e70-b6e9-75a2beadff03.png)

#### Graph View 

- La vue graphique permet de visualiser les dépendances de votre DAG et leur état actuel pour une exécution spécifique.


![image](https://user-images.githubusercontent.com/123757632/231167702-035c7ce5-83f5-4b19-bcff-cdc275425b64.png)

#### Variable View 

- La vue des variables vous permet de lister, créer, éditer ou supprimer les paires clé-valeur d'une variable utilisée pendant les jobs. 
- La valeur d'une variable sera masquée si la clé contient des mots tels que ('password', 'secret', 'passwd', 'authorization', 'api_key', 'apikey', 'access_token') par défaut, mais peut être configurée pour être affichée en clair. 

![image](https://user-images.githubusercontent.com/123757632/231167725-59486066-3838-4aa4-a586-7430d162456d.png)


#### Gantt View 

- Le diagramme de Gantt permet d'analyser la durée et le chevauchement des tâches, et d'identifier rapidement les goulots d'étranglement ainsi que les zones où la majeure partie du temps est dépensée pour des exécutions de DAG spécifiques.

![image](https://user-images.githubusercontent.com/123757632/231167755-11f3e1c4-7a1e-4ed9-8aae-1655790151df.png)


#### Task Duration 

- La durée de chaque tâche pour les N dernières exécutions est affichée dans cette vue. 
- Cela permet de repérer les valeurs aberrantes et de comprendre rapidement où le temps est principalement dépensé dans votre DAG sur plusieurs exécutions.


![image](https://user-images.githubusercontent.com/123757632/231167789-65f8aeec-1b4a-4ab6-95f8-9ed28e9443df.png)

#### Code View

- La transparence est essentielle. 
- Bien que le code de la pipeline soit stocké dans un contrôle de source, cette fonctionnalité permet d'accéder rapidement au code générant le DAG et fournit des informations supplémentaires.


![image](https://user-images.githubusercontent.com/123757632/231167853-ed4d3287-6ab1-416f-a4ec-7044b050b43a.png)